In [ ]:
import os, django
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "medissect.settings")
django.setup()
from django.shortcuts import render
from django.http import HttpResponse, JsonResponse
from explorer.models import ProcedureDescriptor, Provider, Procedure, Location, ProcedureAvgCharges
from django.db.models import Q
import time
import itertools
import sys

In [ ]:
max_n_providers = 200;
max_n_locations = 50;

ne_lat =42.37754562533272
ne_lng =-71.10360633850098
sw_lat =42.36245346811309
sw_lng =-71.13639366149903
code = '99213'
lat = 0.5 * ne_lat + 0.5 * sw_lat;
lng = 0.5 * ne_lng + 0.5 * sw_lng;
d_lat = ne_lat - sw_lat;
d_lng = ne_lng - sw_lng;

time1 = time.time();
unit = '$'
providers = Provider.objects.raw(
  "select \
    explorer_provider.id as id,\
    explorer_provider.npi as npi,\
    explorer_provider.last_name as last_name,\
    explorer_provider.first_name as first_name,\
    explorer_provider.middle_initial as middle_initial,\
    explorer_provider.credentials as credentials,\
    explorer_provider.gender as gender,\
    explorer_provider.is_organization as is_organization,\
    explorer_provider.street2 as street2,\
    explorer_provider.medicare_participant as medicare_participant,\
    explorer_provider.at_facility as at_facility,\
    explorer_provider.location_id as location_id,\
    explorer_procedure.submitted_avg / explorer_procedure.allowed_avg\
      as expensiveness\
  from explorer_procedure\
    join explorer_proceduredescriptor\
      on explorer_procedure.descriptor_id=explorer_proceduredescriptor.id\
    join explorer_provider\
      on explorer_procedure.provider_id = explorer_provider.id\
    join explorer_location\
      on explorer_provider.location_id = explorer_location.id\
  where\
    explorer_proceduredescriptor.code= %(code)s and\
    greatest(\
        abs((explorer_location.latitude -  %(lat0)s) / %(dlat)s),\
        abs((explorer_location.longitude - %(lng0)s) / %(dlng)s)\
      ) < 0.5\
  order by\
    greatest(\
      abs((explorer_location.latitude -  %(lat0)s) / %(dlat)s),\
      abs((explorer_location.longitude - %(lng0)s) / %(dlng)s)\
    ) asc\
  limit %(limit)s;",
  params = {
    "code": code,
    "limit": max_n_providers,
    "lat0": lat, 
    "lng0": lng, 
    "dlat": d_lat, 
    "dlng": d_lng
    }
  )
time2 = time.time();
providers = [p for p in providers];
time3 = time.time();

locations = [list(i2) for i1, i2 in itertools.groupby(providers, lambda x: x.location_id)]

time4 = time.time();

features = [{
"type":"Feature",
"properties": {            
  "providers": [{
    "npi": p.npi, 
    "last_name": p.last_name, 
    "first_name": p.first_name,
    "expensiveness": p.expensiveness} for p in loc],
  "min_expensiveness": min([p.expensiveness for p in loc]),
  "max_expensiveness": max([p.expensiveness for p in loc]),
  "unit": unit
  },
"geometry": {
  "type": "Point", 
  "coordinates": [
    loc[0].location.longitude, 
    loc[0].location.latitude
    ]
  }
} for loc in locations[:max_n_locations]]

time5 = time.time()

response = JsonResponse({
"type": "FeatureCollection",
"features": features,
"procedure": code
})

time6 = time.time()
print "Query: ",    time2 - time1
print "To list: ",  time3 - time2
print "Grouping:",  time4 - time3
print "Formatting", time5 - time4
print "Response",   time6 - time5
print providers

In [ ]:
max_n_providers = 200;
max_n_locations = 50;

ne_lat =42.37754562533272
ne_lng =-71.10360633850098
sw_lat =42.36245346811309
sw_lng =-71.13639366149903
lat = 0.5 * ne_lat + 0.5 * sw_lat;
lng = 0.5 * ne_lng + 0.5 * sw_lng;
d_lat = ne_lat - sw_lat;
d_lng = ne_lng - sw_lng;

time1 = time.time();
unit = ''
providers = Provider.objects.raw(
  "select explorer_provider.*\
  from explorer_provider\
    join explorer_location\
      on explorer_provider.location_id = explorer_location.id\
    where\
      greatest(\
        abs((explorer_location.latitude -  %(lat0)s) / %(dlat)s),\
        abs((explorer_location.longitude - %(lng0)s) / %(dlng)s)\
      ) < 0.5\
    order by\
      greatest(\
        abs((explorer_location.latitude -  %(lat0)s) / %(dlat)s),\
        abs((explorer_location.longitude - %(lng0)s) / %(dlng)s)\
      ) asc\
    limit %(limit)s",
  params = {
    "limit": max_n_providers,
    "lat0": lat, 
    "lng0": lng, 
    "dlat": d_lat, 
    "dlng": d_lng
    }
  );
providers = list(providers);
time2 = time.time();

locations = [list(i2) for i1, i2 in itertools.groupby(providers, lambda x: x.location_id)]

time3 = time.time();

features = [{
"type":"Feature",
"properties": {
  "providers": [{
    "npi": p.npi, 
    "last_name": p.last_name, 
    "first_name": p.first_name,
    "expensiveness": p.expensiveness} for p in loc],
  "min_expensiveness": min([p.expensiveness for p in loc]),
  "max_expensiveness": max([p.expensiveness for p in loc]),
  "unit": unit
  },
"geometry": {
  "type": "Point", 
  "coordinates": [
    loc[0].location.longitude, 
    loc[0].location.latitude
    ]
  }
} for loc in locations[:max_n_locations]]

time4 = time.time()

response = JsonResponse({
#"blah": [p.location.id for p in providers]
"type": "FeatureCollection",
"features": features,
"procedure": code
})

time5 = time.time()
print "Query: ", time2 - time1
print "Grouping:", time3 - time2
print "Formatting", time4 - time3
print "Response", time5 - time4
print providers

SyntaxError: invalid syntax (<ipython-input-25-28249d5193ec>, line 1)